In [1]:
import numpy as np
import pandas as pd

In [2]:
shelter = pd.read_csv("shelterinplace.csv") # scraped via "webscraping.ipynb" notebook, which pulls from NYTimes
data_counties = pd.read_csv("us-counties.csv", dtype={'fips': str}) # pulled from NYTimes github page, https://github.com/nytimes/covid-19-data
city_county = pd.read_csv("city_county.csv") # Manually created to handle city/county discrepancies between two files
demo_county = pd.read_csv("us_county_sociohealth_data.csv", dtype={'fips': str}) # Pulled from github page: https://www.kaggle.com/johnjdavisiv/us-counties-covid19-weather-sociohealth-data

In [3]:
# Convert dates to datetime for comparison
shelter['date_initiated'] = pd.to_datetime(shelter['date'])
data_counties['date'] = pd.to_datetime(data_counties['date'])
shelter.drop('date',axis=1, inplace=True)

# Replace cities with counties
df_join = shelter.merge(city_county, how='left', left_on=['state','area'], right_on=['state','area'])
shelter.loc[df_join['county'].notnull(),'area'] = df_join.loc[df_join['county'].notnull(),'county']

# Remove the " County" suffix from area
shelter['area'] = shelter['area'].str.replace(" County","")

In [4]:
# Join county-level data
county_level = shelter.query('area !="ALL"')

# Merge to county data set
final = data_counties.merge(county_level.drop(['population'],axis=1), how="left", left_on=["county","state"],right_on=["area","state"])

In [5]:
# Join state-level data
state_level = shelter.query('area == "ALL"')

state_subset = final.merge(state_level.drop(['area','population'],axis=1), how="left", left_on="state",right_on="state")

final.loc[final["type"].isnull(),"type"] = state_subset.loc[state_subset['area'].isnull(),'type_y']
final.loc[final["date_initiated"].isnull(),"date_initiated"] = state_subset.loc[state_subset['area'].isnull(),'date_initiated_y']

In [6]:
# Manually add NYC and KSC, which don't have FIPS codes
final.loc[final["county"] == "New York City", "fips"] = "NYC"
final.loc[final["county"] == "Kansas City", "fips"] = "KSC"

# Join in demographic data
demo_county.drop(["state","county"], axis=1, inplace=True)
final = final.merge(demo_county, how="left", left_on=["fips"], right_on=["fips"])

In [7]:
final.head()

,date,county,state,fips,cases,deaths,area,type,date_initiated,lat,...,percentile_rank_minorities,percentile_rank_limited_english_abilities,percentile_rank_minority_status_and_language_theme,percentile_rank_multi_unit_housing,percentile_rank_mobile_homes,percentile_rank_overcrowding,percentile_rank_no_vehicle,percentile_rank_institutionalized_in_group_quarters,percentile_rank_housing_and_transportation,percentile_rank_social_vulnerability
0,2020-01-21,Snohomish,Washington,53061,1,0,NaN,Stay at home,2020-03-23,48.047489,...,0.6784,0.8631,0.8249,0.9472,0.2404,0.7705,0.3473,0.3021,0.5384,0.2961
1,2020-01-22,Snohomish,Washington,53061,1,0,NaN,Stay at home,2020-03-23,48.047489,...,0.6784,0.8631,0.8249,0.9472,0.2404,0.7705,0.3473,0.3021,0.5384,0.2961
2,2020-01-23,Snohomish,Washington,53061,1,0,NaN,Stay at home,2020-03-23,48.047489,...,0.6784,0.8631,0.8249,0.9472,0.2404,0.7705,0.3473,0.3021,0.5384,0.2961
3,2020-01-24,Cook,Illinois,17031,1,0,NaN,Stay at home,2020-03-21,41.840039,...,0.9182,0.9532,0.9631,0.9863,0.0258,0.8491,0.9882,0.4371,0.8742,0.6937
4,2020-01-24,Snohomish,Washington,53061,1,0,NaN,Stay at home,2020-03-23,48.047489,...,0.6784,0.8631,0.8249,0.9472,0.2404,0.7705,0.3473,0.3021,0.5384,0.2961


In [8]:
# Drop unneccessary columns and rows
final.drop("area",axis=1,inplace=True)
final = final.loc[final["county"] != "Unknown"]

# Calculated "Stay at home" binary
conditions = [
    (final['date'] < final['date_initiated']),
    (final['date'] >= final['date_initiated'])]
choices = [0,1]
final['SAH'] = np.select(conditions, choices)
final_cols = ["date",'fips','county', 'state','cases','deaths','type','total_population','population_density_per_sqmi','percent_adults_with_obesity',
              'food_environment_index','income_ratio','median_household_income','percent_65_and_over','percent_less_than_18_years_of_age',
              'percent_non_hispanic_white','life_expectancy','SAH']

final[final_cols].to_csv('final.csv',index=False)
